In [ ]:
import yaml
from pathlib import Path
import pandas as pd

# 1. Define the folder path (use raw string for Windows paths)
folder = Path(r"C:\Users\Admin\Music\stock data")

# 2. Recursively list all files in the folder and subfolders
all_files = [file for file in folder.glob("**/*") if file.is_file()]


header = True  # Write header only once
for file in all_files:
    with open(file, "r") as f:
        data = yaml.safe_load(f)
        df = pd.DataFrame(data)
        df.to_csv(r"C:\Users\Admin\Music\stock data\stock_analysis.csv", mode="a", header=header, index=False)
        print(f"{file} is append")
        header = False  # Skip header after first write


print("Successfully Finished")

In [ ]:
df=pd.read_csv(r"C:\Users\Admin\Music\stock data\stock_analysis.csv")̥
# Group by the 'Ticker' column
grouped = df.groupby('Ticker')

# Loop through each group
for ticker, group_data in grouped:
    # Create a filename based on the ticker
    filename = rf"C:\Users\Admin\Music\stock data\Tickers\{ticker}.csv"
    # Save the group data to CSV
    group_data.to_csv(filename, index=False)
    print(f'{ticker} is saved')



In [ ]:
#""" Clean the csv file and insert it into sql """
""" remove mmonth column , remmove time from the data coloumn """
""" add daily return , sector , cumulativereturn """ 
import pandas as pd
#load the data set

df=pd.read_csv(r"C:\Users\Admin\Music\stock data\stock_analysis.csv")

#Remove duplicate value
df = df.drop_duplicates(subset=["Ticker", "date"])

# Convert to datetime and extract date (ignore time)
df["date"] = pd.to_datetime(df["date"]).dt.date  # Result: 2023-10-

#Delete the mmont column
df.drop("month", axis=1,inplace=True)

#Remove the space
df['Ticker']=df["Ticker"].str.strip()

#confirm the text size
df['Ticker']=df["Ticker"].str.upper()

#sort the column for using groupby
df = df.sort_values(by=["Ticker", "date"])

# 2. Group by Ticker and calculate daily returns
df["Daily_Return"] = (
    df.groupby("Ticker")["close"]
    .pct_change()  # Calculate percentage change from previous day
    .mul(100)       # Convert to percentage
    .round(2)       # Round to 2 decimal places
)

# 3. Handle NaN (first day of data for each Ticker)
df["Daily_Return"] = df["Daily_Return"].fillna(0)  # Optional: Replace NaN with 0%


# Calculate cumulative returns (compounded daily returns)
df["Cumulative_Return"] = (
    df.groupby("Ticker", group_keys=False)["Daily_Return"]  # group_keys=False avoids multi-index
    .apply(lambda x: (1 + x / 100).cumprod() - 1) * 100  # Convert back to percentage
        ) # Convert back to percentage

# Round to 2 decimals
df["Cumulative_Return"] = df["Cumulative_Return"].round(2)

df.to_csv(r"C:\Users\Admin\Music\stock data\stock_cleaned_data.csv",index=False)

In [ ]:
#Load dataset
data=pd.read_csv(r"C:\Users\Admin\Music\stock data\Sector_data.csv")

# Remmove the extra words fromm the symbnol
data['Symbol'] = data['Symbol'].str.split(':').str[-1].str.strip()
# Remove white spaces
data['sector']=data['sector'].str.strip()
data['COMPANY']= data['COMPANY'].str.strip()
#Save the file
data.to_csv(r"C:\Users\Admin\Music\stock data\Sector_data.csv",index=False)

In [ ]:
import pandas as pd

# Mismatched ticker names (old → new)
correction_map = {
    "ADANIGREEN": "ADANIENT",
    "AIRTEL": "BHARTIARTL",
    "TATACONSUMER": "TATACONSUM",
   
}

df = pd.read_csv(r"C:\Users\Admin\Music\stock data\Sector_data.csv")
    
    Replace incorrect symbols
df["Symbol"] = df["Symbol"].replace(correction_map)
    
    # Save corrected CSV
df.to_csv(r"C:\Users\Admin\Music\stock data\Sector_data.csv", index=False)



In [ ]:
import pymysql
import csv


# TiDB Cloud connection parameters
DB_CONFIG = {
    "host": "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    "port": 4000,
    "user": "4V44XYoMA7okY9v.root",
    "password": "LdkgrMO1WG5Tk9Tk",
    "database": "stock_db",
      "ssl_verify_cert": True,
  "ssl_verify_identity" :  True,
  "cursorclass": pymysql.cursors.DictCursor
}

def insert_tickers_data(tickers_csv_path):
    """Insert ticker data with TiDB Cloud connection and SSL"""
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG)
        with connection.cursor() as cursor:
            with open(tickers_csv_path, 'r') as file:
                reader = csv.DictReader(file)
                
                for row in reader:
                    symbol = row['Symbol'].strip()
                    company = row['COMPANY'].strip()
                    sector = row['sector'].strip()

                    # Check existing
                    cursor.execute("SELECT ticker_id FROM tickers WHERE ticker_name = %s", (symbol,))
                    if not cursor.fetchone():
                        cursor.execute("""
                            INSERT INTO tickers (ticker_name, company_name, sector)
                            VALUES (%s, %s, %s)
                        """, (symbol, company, sector))
                        print(f"Inserted: {symbol}")
                        
            connection.commit()
            
    except Exception as e:
        print(f"Error: {str(e)}")
        if connection:
            connection.rollback()
    finally:
        if connection:
            connection.close()
            print("Tickers connection closed")

def insert_stock_data(stock_csv_path):
    """Insert stock data with ticker_id mapping and all required columns"""
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG)
        with connection.cursor() as cursor:
            # Get ticker mapping from database
            cursor.execute("SELECT ticker_id, ticker_name FROM tickers")
            ticker_map = {row['ticker_name']: row['ticker_id'] for row in cursor.fetchall()}

            # Insert stock data
            with open(stock_csv_path, 'r') as file:
                reader = csv.DictReader(file)
                
                for row in reader:
                    # Extract and clean values
                    ticker_name = row['Ticker'].strip()
                    date = row['date']
                    
                    # Skip if ticker not found
                    if ticker_name not in ticker_map:
                        print(f"Skipping: {ticker_name} not found")
                        continue

                    # Prepare data for insertion
                    stock_data = {
                        'ticker_id': ticker_map[ticker_name],
                        'date': date,
                        'open': row.get('open'),
                        'high': row.get('high'),
                        'low': row.get('low'),
                        'close': row.get('close'),
                        'volume': row.get('volume'),
                        'daily_return': row.get('Daily_Return'),
                        'cumulative_return': row.get('Cumulative_Return')
                    }

                    # Convert numeric values
                    for key in ['open', 'high', 'low', 'close', 'volume', 
                               'daily_return', 'cumulative_return']:
                        if stock_data[key]:
                            stock_data[key] = float(stock_data[key])

                    # Check for existing entry
                    cursor.execute("""
                        SELECT data_id FROM stock_data 
                        WHERE ticker_id = %s AND date = %s
                    """, (stock_data['ticker_id'], stock_data['date']))
                    
                    if not cursor.fetchone():
                        # Insert new record
                        cursor.execute("""
                            INSERT INTO stock_data 
                            (ticker_id, date, open, high, low, close, 
                             volume, daily_return, cumulative_return)
                            VALUES (%(ticker_id)s, %(date)s, %(open)s, %(high)s, 
                                    %(low)s, %(close)s, %(volume)s, 
                                    %(daily_return)s, %(cumulative_return)s)
                        """, stock_data)
                        
                        print(f"Inserted: {ticker_name} - {date} - {stock_data['close']}")
            
            connection.commit()
            
    except Exception as e:
        print(f"Error: {str(e)}")
        if connection:
            connection.rollback()
    finally:
        if connection:
            connection.close()
            print("Stock data connection closed")

Execute with your specific file paths
tickers_path = r"C:\Users\Admin\Music\stock data\Sector_data.csv"
stock_path = r"C:\Users\Admin\Music\stock data\stock_cleaned_data.csv"
insert_tickers_data(tickers_path)
insert_stock_data(stock_path)

In [ ]:

# TiDB Cloud connection parameters
DB_CONFIG = {
    "host": "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    "port": 4000,
    "user": "4V44XYoMA7okY9v.root",
    "password": "LdkgrMO1WG5Tk9Tk",
    "database": "stock_db",
      "ssl_verify_cert": True,
  "ssl_verify_identity" :  True,
  "cursorclass": pymysql.cursors.DictCursor
}


def insert_britannia_data(stock_csv_path):
    """Insert Britannia stock data with fixed ticker_id=26"""
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG)
        with connection.cursor() as cursor:
            # Filter and insert Britannia data
            with open(stock_csv_path, 'r') as file:
                reader = csv.DictReader(file)
                britannia_rows = [row for row in reader if row['Ticker'].strip().upper() == 'BRITANNIA']
                
                print(f"Found {len(britannia_rows)} Britannia records to process")
                
                for row in britannia_rows:

                    display(row.get("close"))
                    Prepare data with fixed ticker_id=26
                    stock_data = {
                        'ticker_id': 26,  # Hardcoded based on your ticker table
                        'date': row['date'],
                        'open': float(row.get('open', 0)),
                        'high': float(row.get('high', 0)),
                        'low': float(row.get('low', 0)),
                        'close': float(row.get('close', 0)),
                        'volume': int(float(row.get('volume', 0))),  # Handle decimal volumes
                        'daily_return': float(row.get('Daily_Return', 0)),
                        'cumulative_return': float(row.get('Cumulative_Return', 0))
                    }

                    # Check for existing entry
                    cursor.execute("""
                        SELECT data_id FROM stock_data 
                        WHERE ticker_id = %s AND date = %s
                    """, (stock_data['ticker_id'], stock_data['date']))
                    
                    if not cursor.fetchone():
                        cursor.execute("""
                            INSERT INTO stock_data 
                            (ticker_id, date, open, high, low, close, 
                             volume, daily_return, cumulative_return)
                            VALUES (%(ticker_id)s, %(date)s, %(open)s, %(high)s, 
                                    %(low)s, %(close)s, %(volume)s, 
                                    %(daily_return)s, %(cumulative_return)s)
                        """, stock_data)
                        
                        print(f"Inserted Britannia - {stock_data['date']} - Close: {stock_data['close']}")
            
            connection.commit()
            
    except Exception as e:
        print(f"Error: {str(e)}")
        if connection:
            connection.rollback()
    finally:
        if connection:
            connection.close()
            print("Britannia data connection closed")

# Execute with your path
stock_path = r"C:\Users\Admin\Music\stock data\stock_cleaned_data.csv"
insert_britannia_data(stock_path)